In [ ]:
!pip install fastapi
!pip install colabcode
!pip install h2o -qq
!npm install -g localtunnel -qq > /dev/null

In [ ]:
import pandas as pd
from fastapi import FastAPI
import pickle

app = FastAPI()

@app.get('/')
def index():
    data = pd.read_excel("/content/drive/MyDrive/ISB_AMPBA(2021-2022)/Term- 5/FP-2/oil_balance.xlsx")
    data= data.drop(columns=['GDEBTSA','PPISA', 'PRCUS', 'PUCUS'])
    data['oilprod_week_avg'] = data['INDPROD'].rolling(7).mean()
    data['oilprod_2weeks_avg'] = data['INDPROD'].rolling(14).mean()
    data['oilprod_month_avg'] = data['INDPROD'].rolling(30).mean()
    #Extracting year,month and date from the Date field.
    data['year'] = data['Date'].dt.year
    data['month'] = data['Date'].dt.month
    data['day'] = data['Date'].dt.day
    data.rename(columns = {'INDPROD':'int_prod','REFINOBS':'ref_intake','TOTEXPSB':'total_export','GDEBTSA':'govt_debt','EXPSA':'exp_sa','IPSA':'imp_sa','PRCUS':'pvtcons_us',
                          'PUCUS':"pubcons_us",'REFINOBS':'ref_intake','OILPRODSA':'saudi_oil_prod','OILPRODUS':'usa_oil_prod'},inplace = True)
    data = data[data.Date.notnull()]
    data.reset_index(drop=True,inplace=True)
    train = data.loc[:int(data.shape[0]*0.8),:]
    test = data.loc[int(data.shape[0]*0.8):,:]
    import h2o
    h2o.init(nthreads=-1)
    model_path = '/content/drive/MyDrive/ISB_AMPBA(2021-2022)/Term- 5/FP-2/XGBoost_grid_1_AutoML_1_20220716_175707_model_312' 
    model = h2o.load_model(model_path)
    hf_train = h2o.H2OFrame(train)
    hf_test = h2o.H2OFrame(test)
    hf_test_predict = model.predict(hf_test)
    hf_test_predict = h2o.as_list(hf_test_predict)
    return hf_test_predict.head()

@app.post('/Predict')     
async def forecast():
  data = pd.read_excel("/content/drive/MyDrive/ISB_AMPBA(2021-2022)/Term- 5/FP-2/oil_balance.xlsx")
  data= data.drop(columns=['GDEBTSA','PPISA', 'PRCUS', 'PUCUS'])
  data['oilprod_week_avg'] = data['INDPROD'].rolling(7).mean()
  data['oilprod_2weeks_avg'] = data['INDPROD'].rolling(14).mean()
  data['oilprod_month_avg'] = data['INDPROD'].rolling(30).mean()
  #Extracting year,month and date from the Date field.
  data['year'] = data['Date'].dt.year
  data['month'] = data['Date'].dt.month
  data['day'] = data['Date'].dt.day
  data.rename(columns = {'INDPROD':'int_prod','REFINOBS':'ref_intake','TOTEXPSB':'total_export','GDEBTSA':'govt_debt','EXPSA':'exp_sa','IPSA':'imp_sa','PRCUS':'pvtcons_us',
                        'PUCUS':"pubcons_us",'REFINOBS':'ref_intake','OILPRODSA':'saudi_oil_prod','OILPRODUS':'usa_oil_prod'},inplace = True)
  data = data[data.Date.notnull()]
  data.reset_index(drop=True,inplace=True)
  train = data.loc[:int(data.shape[0]*0.8),:]
  test = data.loc[int(data.shape[0]*0.8):,:]
  import h2o
  h2o.init(nthreads=-1)
  model_path = '/content/drive/MyDrive/ISB_AMPBA(2021-2022)/Term- 5/FP-2/XGBoost_grid_1_AutoML_1_20220716_175707_model_312' 
  model = h2o.load_model(model_path)
  hf_train = h2o.H2OFrame(train)
  hf_test = h2o.H2OFrame(test)
  hf_test_predict = model.predict(hf_test)
  return hf_test_predict.head(10)


In [ ]:
from colabcode import ColabCode
server = ColabCode(port=5000, code=False)

In [ ]:
server.run_app(app=app)